In [ ]:
import tensorflow as tf
import argparse
import os.path
from cifar100_models import *
from cifar100_input import *
from utils import *
import itertools
import logging
import pickle

tf.logging.set_verbosity(tf.logging.INFO)

# get TF logger
log = logging.getLogger('tensorflow')
log.setLevel(logging.INFO)

# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# create file handler which logs even debug messages
fh = logging.FileHandler('tensorflow_cifar100.log')
fh.setLevel(logging.INFO)
fh.setFormatter(formatter)
log.addHandler(fh)

#### SET THIS BEFORE RUNNING
DIR = "cifar100"
NAME = "cifar100_scm_{}_{}_{}_{}"
PATH = os.path.join(DIR,NAME)

def train(model, name, restore_point = None, wd1=0., wd2=0., wd3=0., drop1=1., save=True):
    with tf.Session() as sess:
        model._restore_or_init(sess, restore_point)
        model.weight_decay_layer_1.assign(wd1)
        model.weight_decay_layer_2.assign(wd2)
        model.weight_decay_layer_3.assign(wd3)
        model.drop_fc1.assign(drop1)
        
        try:
            while True:
                sess.run(model.train_step)
        except tf.errors.OutOfRangeError as e:
            pass
        
        global_step = model.global_step.eval()
        if save:
            checkpoint_name = CHECKPOINT.format(name) + "_" + str(global_step).rjust(10,"0")
            tf.logging.info("Saving to: " + checkpoint_name)
            model.saver.save(sess, checkpoint_name)
    return checkpoint_name
        
def evaluate(model, restore_point = None):
    acc = []
    with tf.Session() as sess:
        model._restore_or_init(sess, restore_point)
        try:
            while True:
                acc.append(model.accuracy.eval())
        except tf.errors.OutOfRangeError as e:
            pass
    return np.mean(acc)

@np.vectorize
def nplte(x,y):
    return x<=y

def train_to_convergence(train_files, test_files, wd1=0., wd2=0., wd3=0., drop1=1., save_rate=10):
    acc = []
    name = PATH.format(wd1,wd2,wd3,drop1)
    make_dirs(CHECKPOINT.format(name))
    checkpoint = latest_checkpoint(name)
    i = 0
    while len(acc)<30 or nplte(np.array(acc[-20:]) - np.array(acc[-21:-1]),0).sum() < 13: # until the accuracy is just as likely to go down as up
        tf.reset_default_graph()
        train_data = Cifar100Record(train_files, epochs=1)
        model = Cifar100ShallowConvolutionalModel(train_data.image, train_data.label)
        acc.append(evaluate(model, restore_point = checkpoint))
        tf.reset_default_graph()
        train_data = Cifar100Record(train_files, epochs=1)
        model = Cifar100ShallowConvolutionalModel(train_data.image, train_data.label)
        save = True
        checkpoint = train(model, name, restore_point = checkpoint, wd1=wd1, wd2=wd2, wd3=wd3, drop1=drop1, save=save)
        tf.logging.info(acc[-21:])
        i+=1
    return acc
        
def train_a_bunch_of_cifar100s():
    wd1_space = np.linspace(0.,0.012,3) # they use 0.004 in the tensorflow example
    wd2_space = np.linspace(0.,0.012,3)
    wd3_space = np.linspace(0.,0.012,3)
    drop1_space = np.linspace(0.2,1,3)
    wd_drop_grid = itertools.product(wd1_space, wd2_space, wd3_space, drop1_space)
    train_files, test_files, validation_files = maybe_download_and_extract()
    try:
        with open("completed_cifar100.p",'rb') as f:
            completed = pickle.load(f)
    except:
        completed = []
    i = 0
    for wd1,wd2,wd3,drop1 in wd_drop_grid:
        if i < len(completed): #skip values until we get to a new one
            i+=1
            continue
        acc = train_to_convergence(train_files,test_files,wd1,wd2,wd3,drop1)
        tf.logging.info(acc[-10:])
        completed.append((wd1,wd2,wd3,drop1,acc))
        i+=1
        with open("completed_cifar100.p","wb") as f:
            pickle.dump(completed,f)

In [ ]:
make_dirs(ROOT.format(DIR))
train_a_bunch_of_cifar100s()

INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.2/model.ckpt_0000215600
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.2/model.ckpt_0000215600
INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.2/model.ckpt_0000216000
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.2/model.ckpt_0000216000 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[0.99942499]
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.2/model.ckpt_0000216000
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.2/model.ckpt_0000216000
INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.2/model.ckpt_0000216400
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.2/model.ckpt_0000216400 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[0.999424

INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.2/model.ckpt_0000221200
INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.2/model.ckpt_0000221600
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.2/model.ckpt_0000221600 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[0.99942499, 0.99947494, 0.99952501, 0.99947494, 0.99949998, 0.99955004, 0.99955004, 0.99955004, 0.99959999, 0.99962497, 0.99964994, 0.99964994, 0.99967498, 0.99970001, 0.99977493]
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.2/model.ckpt_0000221600
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.2/model.ckpt_0000221600
INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.2/model.ckpt_0000222000
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.2/model.ckpt_0000222000 is not in all_model_checkpoin

INFO:tensorflow:[0.99955004, 0.99955004, 0.99955004, 0.99959999, 0.99962497, 0.99964994, 0.99964994, 0.99967498, 0.99970001, 0.99977493, 0.99970001, 0.99977493, 0.99974996, 0.99972504, 0.99979997, 0.99979997, 0.99979997, 0.99985003, 0.999825, 0.999825, 0.999825]
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.2/model.ckpt_0000226000
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.2/model.ckpt_0000226000
INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.2/model.ckpt_0000226400
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.2/model.ckpt_0000226400 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[0.99955004, 0.99955004, 0.99959999, 0.99962497, 0.99964994, 0.99964994, 0.99967498, 0.99970001, 0.99977493, 0.99970001, 0.99977493, 0.99974996, 0.99972504, 0.99979997, 0.99979997, 0.99979997, 0.99985003, 0.999825, 0.999825, 0.999825, 0.99985003]
I

INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.6000000000000001/model.ckpt_0000003200
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.6000000000000001/model.ckpt_0000003200 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[0.0082999989, 0.030400001, 0.046475001, 0.062475, 0.072749995, 0.089300007, 0.095725, 0.10895]
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.6000000000000001/model.ckpt_0000003200
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.6000000000000001/model.ckpt_0000003200
INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.6000000000000001/model.ckpt_0000003600
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.6000000000000001/model.ckpt_0000003600 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[0.0082999989, 0.030400001, 0.046475001, 0.062475, 0.072749995, 

INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.6000000000000001/model.ckpt_0000007600
INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.6000000000000001/model.ckpt_0000008000
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.6000000000000001/model.ckpt_0000008000 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[0.0082999989, 0.030400001, 0.046475001, 0.062475, 0.072749995, 0.089300007, 0.095725, 0.10895, 0.11827502, 0.1292, 0.13645001, 0.14109999, 0.15035, 0.1559, 0.16362499, 0.17040001, 0.17625, 0.18020001, 0.18865001, 0.191825]
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.6000000000000001/model.ckpt_0000008000
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.6000000000000001/model.ckpt_0000008000
INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.012_0.6000000000000001/model.

...

INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000024400
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000024400
INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000024800
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000024800 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[0.26519999, 0.26399997, 0.27127501, 0.26989999, 0.2757, 0.27489999, 0.275525, 0.28224999, 0.28364998, 0.28974998, 0.290575, 0.29427499, 0.29892498, 0.29817498, 0.30289999, 0.30344999, 0.30539998, 0.31082496, 0.31274998, 0.31387496, 0.31672499]
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000024800
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000024800
INFO:tensorflow:Saving to: ../models/c

INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000029200
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000029200 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[0.29427499, 0.29892498, 0.29817498, 0.30289999, 0.30344999, 0.30539998, 0.31082496, 0.31274998, 0.31387496, 0.31672499, 0.31865001, 0.323275, 0.32049999, 0.32939997, 0.3337, 0.33134997, 0.3346, 0.3382, 0.33594999, 0.34087503, 0.33465001]
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000029200
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000029200
INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000029600
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000029600 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[0.29892498

INFO:tensorflow:[0.323275, 0.32049999, 0.32939997, 0.3337, 0.33134997, 0.3346, 0.3382, 0.33594999, 0.34087503, 0.33465001, 0.34007499, 0.34837502, 0.34595001, 0.35124999, 0.3511, 0.35852501, 0.35782501, 0.36025003, 0.36620003, 0.36550003, 0.36020002]
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000033600
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000033600
INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000034000
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000034000 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[0.32049999, 0.32939997, 0.3337, 0.33134997, 0.3346, 0.3382, 0.33594999, 0.34087503, 0.33465001, 0.34007499, 0.34837502, 0.34595001, 0.35124999, 0.3511, 0.35852501, 0.35782501, 0.36025003, 0.36620003, 0.36550003, 0.36020002, 0.37110001]
INFO:tensorflow:Restori

INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000038000
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000038000
INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000038400
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000038400 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[0.34595001, 0.35124999, 0.3511, 0.35852501, 0.35782501, 0.36025003, 0.36620003, 0.36550003, 0.36020002, 0.37110001, 0.36552498, 0.37377504, 0.37467498, 0.38027501, 0.37367499, 0.38237503, 0.38745004, 0.3777, 0.38464999, 0.38510001, 0.39052498]
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000038400
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000038400
INFO:tensorflow:Saving to: ../models/c

INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000042800
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000042800 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[0.37377504, 0.37467498, 0.38027501, 0.37367499, 0.38237503, 0.38745004, 0.3777, 0.38464999, 0.38510001, 0.39052498, 0.393675, 0.39757499, 0.3994, 0.39887497, 0.40277499, 0.40627503, 0.40764999, 0.402825, 0.40544999, 0.413425, 0.41629997]
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000042800
INFO:tensorflow:Restoring parameters from ../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000042800
INFO:tensorflow:Saving to: ../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000043200
INFO:tensorflow:../models/cifar100/cifar100_scm_0.0_0.0_0.012_1.0/model.ckpt_0000043200 is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:[0.37467498